In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pathlib
import pandas as pd
import re
import polars as pl

import matplotlib.pyplot as plt
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
plt.rcParams['svg.fonttype'] = 'none'

import sys
from alada import chap01 as ch01
from alada import casestudy01 as cs01

# Create data folder for the case study
cs01.create_data_folder()

# Case Study 01: Clustering of Doctors' Notes
---

The first case study we will be looking at as part of the ALADA couse is the problem of clustering medical notes using measures of similarities between vectors. We make use of a kaggle dataset for this purpose. The first step to running this notebook is to download the dataset and copy it into the data directory `data/case_study_01/`. 

Link to the kaggle dataset: [https://www.kaggle.com/datasets/gauravmodi/doctors-notes/data](https://www.kaggle.com/datasets/gauravmodi/doctors-notes/data)

Download the reports.csv file, and copy it into the folder `data/case_study_01/`. After you have done this, run the following cell and check if it reports success.

In [3]:
cs01.check_dataset()

'Success! You can run this notebook.'

## 1. What does this dataset have?

The data file is a `.csv` file. If you do not know that means, then take a look [here](https://en.wikipedia.org/wiki/Comma-separated_values).

The file has two columns: `medical_specialty` and `report`.
- The `medical_specialty` column contains the specialty of the doctor who wrote the report, and
- the `report` column contains the actual text of the report written down by the doctor for a particular patient.

For the purpose of this case study, we will only be using the `report` column; we will use the `medical_specialty` column only for evaluating the clustering results for us know how well out clustering algorithm has done.

You are probably asking at this point, we have learned about clustering using n-vectors; objects containing numbers. How does one cluster text data? We do that by first converting text data into vectors that represent, at least partially, the information contaning in the text. There are many ways to do this, but will make use a simple technique called `count vectorization`.

**Count Vectorization** is a technique that converts text data into vectors by counting the number of times a set of `tokens` (chosen words) appears in the text. Tokens are words that are chosen from the text or given by a specialist from the field of study. Consider the following example:

**Text to convert:** "The quick brown fox jumps over the lazy dog. The dog is very lazy, but the fox is quick. The dog is friendly through."

**Tokens:** ["quick", "brown", "fox", "jumps", "lazy", "dog"]

We simply count the number of times each token appears in the text. The vector representation of the text is then: [2, 1, 2, 1, 2, 3]


## 1. Clustering medical notes

This problem deals with clsutering medical notes based on the test content. The 
original dataset is from Kaggle repository: https://www.kaggle.com/c/medical-notes

This data is in the file `data/reorts.csv`.

The data comes in a simple CSV format with two columns: `medical_specialty` and `report`.

I have added a header to this file by generating a unique set of tokens identified from the original data set.

The intial subsection has the unique token generation code, and the following subsection performs the clustering by ignoring the actual `medical_specialty` column.

### 1.1 Generate unique tokens

This is quite simple. We collect reports from each `medical_speciality` and makesa list of all unique words. Following this, only the words that unique occur in each speciality are retained, and then rest are dropped.

#### Supporting Functions

In [2]:
def count_tokens(note: str, tokens: list) -> np.array:
    """Find the token count in the medical records."""
    return np.array([note.count(tok) for tok in tokens])
ss
def get_all_text_for_medspec(medrec: pl.DataFrame, col: str) -> dict:
    """Combine all text for each medical specialty."""
    return {k: ' '.join(v[col].values) for k, v in medrec.groupby("medical_specialty")}

def get_tokens_for_medspec(medrec: pl.DataFrame, col: str, minlen: int=5) -> dict:
    """Get the tokens of length greater than minlen for each medical specialty."""
    pattern = re.compile(r'[^\w\s]|\d')
    alltext = get_all_text_for_medspec(medrec, col)
    return {
        ms: [s for s in np.unique(txt.split()) if not pattern.search(s) and len(s) > minlen]
        for ms, txt in alltext.items()
    }

def get_unique_tokens_for_medspec(ms_tokens: dict) -> dict:
    """Get unique tokens for each medical specialty."""
    ms_unique_tokens = {}
    for _k, _v in ms_tokens.items():
        # Find the unique tokens for each medical specialty
        _toks = set(_v)
        for _ms in ms_tokens.keys():
            if _ms != _k:
                _toks -= set(ms_tokens[_ms])
        ms_unique_tokens[_k] = list(_toks)
    return ms_unique_tokens

ErrorException: syntax: extra token "count_tokens" after end of expression

#### Code to generate unique tokens

In [3]:
# Read header for tokens
datafile = (datadir / pathlib.Path("reports.csv")).as_posix()
with open(datafile, "r") as f:
    tokens = [_s.strip() for _s in f.readline().split(':')[-1].split(",")]

# Read the medical records file.
medrec = pd.read_csv(datafile, skiprows=2)
medrec["medical_specialty"].value_counts()
# Make the reports column lower case
medrec["report_lower"] = medrec["report"].str.lower()

UndefVarError: UndefVarError: pathlib not defined

In [4]:
# Get all potential tokens for each medical specialty
ms_tokens = get_tokens_for_medspec(medrec, col="report_lower")
ms_unique_tokens = get_unique_tokens_for_medspec(ms_tokens)

UndefVarError: UndefVarError: get_tokens_for_medspec not defined

In [5]:
# All text for each medical specialty
ms_alltext = get_all_text_for_medspec(medrec, col="report_lower")
ms_token_count = {}
for k, txt in ms_alltext.items():
    _cnts = [txt.count(_t) for _t in ms_unique_tokens[k]]
    # Polar dataframe from the counts dictionary
    ms_token_count[k] = pl.DataFrame(
        data={"token": ms_unique_tokens[k], "count": _cnts},
        schema={"token": pl.String, "count": pl.Int64}
    ).sort("count", descending=True)

# Choose the top 15 tokens for each medical specialty
top_tokens = []
for k, v in ms_token_count.items():
    top_tokens += v.head(5)["token"].to_list()

UndefVarError: UndefVarError: get_all_text_for_medspec not defined

### 1.2 Clustering medical rpoets through `kmeans`

We now assume that we only have access to the medical reports and not to the speciality they correspond to. We make use of the unique tokens to count the frequency of each of these tokens in each report to generate a set of column vectors for each report. This column vector is then used to compare the similarity between reports, and cluster them.

There are many entries from the surgery medical speciality, and thus this will be removed from the data for the clustering.

In [6]:
datadir = pathlib.Path("data")
outdir = pathlib.Path("output")
outdir.mkdir(exist_ok=True)

UndefVarError: UndefVarError: pathlib not defined

In [7]:
# Read header for tokens
datafile = (datadir / pathlib.Path("reports.csv")).as_posix()
with open(datafile, "r") as f:
    tokens = [_s.strip() for _s in f.readline().split(':')[-1].split(",")]

# Read the medical records file.
medrec = pd.read_csv(datafile, skiprows=2)
medrec["medical_specialty"].value_counts()
# Make the reports column lower case
medrec["report_lower"] = medrec["report"].str.lower()

UndefVarError: UndefVarError: pathlib not defined

In [8]:
# Get the token vectors for each medical note.
inx = medrec["medical_specialty"] != "Surgery"
medrec = medrec[inx]
medrec.reset_index(inplace=True, drop=True)
tokvec = np.array([count_tokens(_v, tokens) for _v in medrec['report_lower'].values])
tokvec.shape

UndefVarError: UndefVarError: medrec not defined

In [21]:
k = 4
km = chap01.KMeans(X=tokvec, k=4)
cm, ca, j = km.fit()

In [ ]:
# Find the medical specialty for each cluster
medrec["cluster"] = ca[-1]
# Print value counts for different clusters.
for _c in range(k):
    print(f"Cluster {_c}: {np.linalg.norm(cm[-1][_c]): .3f}")
    print(medrec[medrec["cluster"] == _c]["medical_specialty"].value_counts())
    print()

In [ ]:
plt.plot(cm[-1].T + np.array([0, 1, 2, 3]));

### Clustering using k-means


In [ ]:
def get_cluster_assignment(tvec, mvec):
    return np.argmin(np.linalg.norm(mvec - tvec, axis=1))

def get_cluster_mean(tokvec, clustassign):
    return np.array([np.mean(tokvec[clustassign == _c, :], axis=0)
                     for _c in np.unique(clustassign)])

def get_j_clust(tokvec, mvec, clustassign):
    return np.sum([np.sum(np.square(np.linalg.norm(tokvec[clustassign == _c, :] - mvec[_i, :], axis=1))) for _i, _c in enumerate(np.unique(clustassign))])


In [ ]:
k = 3
# Random starting means.
mvec = tokvec[list(np.random.randint(0, tokvec.shape[0], k)), :]

# Cluster assingment for the random means.
clustassign = np.array([get_cluster_assignment(_tvec, mvec) for _tvec in tokvec])

# Compute J_clust
J_clust_curr = get_j_clust(tokvec, mvec, clustassign)

# Iterate now.
n_iter = 20
for i in range(n_iter):
    # Update the means.
    mvec = get_cluster_mean(tokvec, clustassign)
    
    # Update the cluster assignment.
    clustassign = np.array([get_cluster_assignment(_tvec, mvec) for _tvec in tokvec])
    
    # Update J_clust
    J_clust_prev = J_clust_curr
    J_clust_curr = get_j_clust(tokvec, mvec, clustassign)

    print(f"Iteration: {i:2d}, J_clust: {J_clust_curr:6.2f}, Change: {100 * (J_clust_curr - J_clust_prev) / J_clust_prev:.2f}%")

In [ ]:
# Functions for performing k-means algorithm 
def get_cluster_assignment(tvec, mvec):
    return np.argmin(np.linalg.norm(mvec - tvec, axis=1))

def get_cluster_mean(tokvec, clustassign):
    return np.array([np.mean(tokvec[clustassign == _c, :], axis=0)
                     for _c in np.unique(clustassign)])

def get_j_clust(tokvec, mvec, clustassign):
    return np.sum([np.sum(np.square(np.linalg.norm(tokvec[clustassign == _c, :] - mvec[_i, :], axis=1))) for _i, _c in enumerate(np.unique(clustassign))])


### Demo of kmean class from chap01 module

In [ ]:
# Generate sample data
_x1 = np.random.randn(100, 2) + np.array([4, 4])
_x2 = np.random.randn(100, 2)
X = np.vstack([_x1, _x2])
# Randomly reorder the rows
np.random.shuffle(X)

In [ ]:
k=2
km = chap01.KMeans(X, 2)
cm, ca, j = km.fit()

In [ ]:
# Plot the k-mean algorithm evolution.
colors = ["tab:blue", "tab:red", "tab:green", "tab:orange", "tab:purple", "tab:brown", "tab:pink", "tab:gray", "tab:olive", "tab:cyan"]
n = len(cm)
m = (n // 5) + 1
fig = plt.figure(figsize=(15, 2.5 * m))
for i in range(n):
    ax = fig.add_subplot(m, 5, i + 1)
    for _k in range(k):
        ax.scatter(X[ca[i] == _k, 0], X[ca[i] == _k, 1], s=25, color=colors[_k],
                   marker="x", alpha=0.2)
    for _k in range(k):
        ax.scatter(cm[i][_k, 0], cm[i][_k, 1], s=50, marker="o", alpha=1, 
                   color=colors[_k], edgecolors="black") 
    ax.set_title(f"Iter: {i + 1}: J = {j[i]:.1f}")

#### Similarity problem

In [ ]:
X = np.array([
    [167, 102, 36.6],
    [180, 87, 26.9],
    [177, 78, 24.9],
    [152, 76, 32.9],
]).T

# Compute the distance between the points.
np.array([[np.linalg.norm(_v1 - _v2) for _v2 in X.T] for _v1 in X.T])

In [ ]:
X1 = X * np.array([0.01, 1, 1]).reshape(-1, 1)
# Compute the distance between the points.
np.array([[np.linalg.norm(_v1 - _v2) for _v2 in X1.T] for _v1 in X1.T])

##### Angle similarity

In [ ]:
def angle_between(v1, v2):
    return (180 / np.pi) * np.arccos(np.max([-1, np.min([1, np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))])]))

# Compute the distance between the points.
np.round(100 * np.array([[angle_between(_v1, _v2) for _v2 in X.T] for _v1 in X.T])) / 100

In [ ]:
# Compute the distance between the points.
np.round(100 * np.array([[angle_between(_v1, _v2) for _v2 in X1.T] for _v1 in X1.T])) / 100

In [ ]:
X

In [ ]:
X1

In [ ]:
np.round(100 * np.array([v / np.linalg.norm(v) for v in X.T]).T) / 100

In [ ]:
np.round(100 * np.array([v / np.linalg.norm(v) for v in X1.T]).T) / 100